In [208]:
import cv2
import pandas as pd
import glob

In [209]:
sign_alphabet = [
    *glob.glob("data/asl_alphabet_train/**/*.jpg"),

]
sign_alphabet

['data/asl_alphabet_train/R/R2837.jpg',
 'data/asl_alphabet_train/R/R2189.jpg',
 'data/asl_alphabet_train/R/R1480.jpg',
 'data/asl_alphabet_train/R/R1494.jpg',
 'data/asl_alphabet_train/R/R2823.jpg',
 'data/asl_alphabet_train/R/R228.jpg',
 'data/asl_alphabet_train/R/R200.jpg',
 'data/asl_alphabet_train/R/R566.jpg',
 'data/asl_alphabet_train/R/R572.jpg',
 'data/asl_alphabet_train/R/R214.jpg',
 'data/asl_alphabet_train/R/R1325.jpg',
 'data/asl_alphabet_train/R/R1443.jpg',
 'data/asl_alphabet_train/R/R599.jpg',
 'data/asl_alphabet_train/R/R1457.jpg',
 'data/asl_alphabet_train/R/R1331.jpg',
 'data/asl_alphabet_train/R/R2638.jpg',
 'data/asl_alphabet_train/R/R1319.jpg',
 'data/asl_alphabet_train/R/R2610.jpg',
 'data/asl_alphabet_train/R/R2176.jpg',
 'data/asl_alphabet_train/R/R2162.jpg',
 'data/asl_alphabet_train/R/R2604.jpg',
 'data/asl_alphabet_train/R/R1682.jpg',
 'data/asl_alphabet_train/R/R980.jpg',
 'data/asl_alphabet_train/R/R758.jpg',
 'data/asl_alphabet_train/R/R1696.jpg',
 'data/a

In [210]:

pics = dict()
for foto in sign_alphabet:
    label = foto.split("/")[-2]
    pic = foto
    if label in pics:
        pics[label].append(pic)
    else:
        pics[label] = [pic]

In [211]:
im = []
for k,v in pics.items():
    for a in v:
        im.append([k,a])
alphabet =  pd.DataFrame(im, columns=["label","foto"])
alphabet = alphabet.sort_values(by='label',)
alphabet.head()


alphabet[alphabet.label=='A'].head()

,label,foto
24000,A,data/asl_alphabet_train/A/A25.jpg
25995,A,data/asl_alphabet_train/A/A933.jpg
25996,A,data/asl_alphabet_train/A/A2375.jpg
25997,A,data/asl_alphabet_train/A/A932.jpg
25998,A,data/asl_alphabet_train/A/A2412.jpg


In [313]:
letters = alphabet.label.unique()
filtered_df = pd.DataFrame()
for letter in letters:
    filtered_df= filtered_df.append(alphabet[alphabet.label==letter].head(1000))
filtered_df.label.value_counts()

O          1000
H          1000
W          1000
E          1000
K          1000
Q          1000
M          1000
X          1000
P          1000
U          1000
S          1000
F          1000
I          1000
A          1000
D          1000
Y          1000
V          1000
R          1000
C          1000
N          1000
J          1000
L          1000
T          1000
Z          1000
G          1000
del        1000
space      1000
B          1000
nothing    1000
Name: label, dtype: int64

In [314]:
def normalize_image(picFile):
    dim = (100,100)
    im = cv2.imread(picFile)    
    
    im_bw = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
#     plt.imshow(im_bw, cmap="gray")
    im_bw = cv2.resize(im_bw, dim, interpolation=cv2.INTER_AREA)
#     ret,binary_image = cv2.threshold(im_bw,60,255,cv2.THRESH_BINARY)

    return im_bw/255

In [ ]:
filtered_df["normalized_image"] = filtered_df.foto.apply(lambda path: normalize_image(path))
filtered_df["normalized_image"].head()

In [ ]:
train_images = filtered_df['normalized_image']
train_labels = filtered_df['label']

X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2) 
le = preprocessing.LabelEncoder()
le.fit(letters)
labels =le.transform(y_train)
train_labels = keras.utils.to_categorical(labels, num_classes =30)


# y_trainHot = to_categorical(y_train, num_classes = 30)
# y_testHot = to_categorical(y_test, num_classes = 30)
X_train = np.stack(X_train, axis=0) 
X_train = X_train.reshape(X_train.shape[0],100,100,1)
print(X_train.shape)
print(train_labels.shape)


In [ ]:


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                        input_shape=(100, 100, 1))) 

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(30, activation='softmax')) # 10 clases


model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, train_labels, epochs=10, batch_size=64)


In [101]:
from keras.callbacks import ModelCheckpoint

# num_of_classes = get_num_of_classes()
model = Sequential()
model.add(Conv2D(16, (2,2), input_shape=(100,100, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
sgd = optimizers.SGD(lr=1e-2)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
filepath="cnn_model_keras2.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]
#from keras.utils import plot_model
#plot_model(model, to_file='model.png', show_shapes=True)
# return model, callbacks_list

In [102]:
model.fit(train_images, train_labels, epochs=4, batch_size=128)


Epoch 1/4
4000/4000 [==============================] - 23s 6ms/step - loss: 12.1109 - acc: 0.2465
Epoch 2/4
4000/4000 [==============================] - 22s 6ms/step - loss: 12.0886 - acc: 0.2500
Epoch 3/4
4000/4000 [==============================] - 24s 6ms/step - loss: 12.0886 - acc: 0.2500
Epoch 4/4
4000/4000 [==============================] - 37s 9ms/step - loss: 12.0886 - acc: 0.2500


In [265]:
from keras.applications.vgg16 import VGG16

weight_path1 = 'keras_pre_trained_models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model_1 = VGG16(weights = 'imagenet', include_top=False, input_shape=(100,100, 3))

In [266]:
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
optimizer1 = keras.optimizers.Adam()

# train_labels.shape
pretrained_model_1.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer1, 
                  metrics=['accuracy'])
train_labels.shape

(3200, 4)

In [222]:
# map_characters1 = map_characters
import tensorflow as tf

from sklearn.utils import class_weight
class_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
weight_path1 = './keras_pre_trained_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

pretrained_model_1 = VGG16(weights = weight_path1, include_top=False, input_shape=(100, 100, 3))
#pretrained_model_2 = InceptionV3(weights = weight_path2, include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = keras.optimizers.Adam()
optimizer2 = keras.optimizers.RMSprop(lr=0.0001)
def pretrainedNetwork(xtrain,ytrain,xtest,ytest,pretrainedmodel,pretrainedweights,classweight,numclasses,numepochs,optimizer):
    base_model = pretrained_model_1 # Topless
    # Add top layer
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(numclasses, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
    # Train top layer
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    model.summary()
    # Fit model
    history = model.fit(xtrain,ytrain, epochs=numepochs, class_weight=classweight,
                        validation_data=(xtest,ytest), verbose=1,callbacks = [MetricsCheckpoint('logs')])
    # Evaluate model
    score = model.evaluate(xtest,ytest, verbose=0)
    print('\nKeras CNN - accuracy:', score[1], '\n')

    return model
pretrainedNetwork(X_train, y_trainHot, X_test, y_testHot,pretrained_model_1,weight_path1,class_weight1,30,10,optimizer1)

TypeError: Checkpointable._track_checkpointable() passed type <class 'keras.engine.input_layer.InputLayer'>, not a Checkpointable.

In [129]:
train_images.shape

(4000,)